**Approach 1.1:**

**Training:**

-> For every selected frame of video, extract features and give label(label generated by converting manually marked safe duration into frame intervals)

-> Use classification to predict 0/1 for each frame

**Testing :**

For every frame, extract features and use classification to predict label

**Features to be extracted from individual frames:**

1. Number of bounding boxes in every region (total region:6)
2. Total Area covered by all bounding boxes in every region
3. Minimum distance of bounding box from every region

In [61]:
#importing necessary libraries
from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
%matplotlib inline  
from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from sklearn.svm import SVC 
from sklearn.metrics import classification_report, confusion_matrix 
from sklearn.model_selection import GridSearchCV
import sys
import cv2
from random import randint


In [62]:
name = 'siddhi'

if name == 'siddhi':
    path_train_arrays = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_train_v2'
    path_test_arrays = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_test_v2'
    path_train_labels = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_train.csv'
    path_test_labels = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/labels_test.csv'
    path_train_videos = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_train'
    path_test_videos = '/home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/videos_test'


elif name == 'yagnesh':
    path_train_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_train_v2'
    path_test_arrays = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/arrays_test_v2'
    path_train_labels = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_train.csv'
    path_test_labels = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_test.csv'

In [63]:
# for distance calculation for every region
bottom_center_point_x = 1920/2    # width/2
bottom_center_point_y = 1080      # height
cx = 1865
cy = 320

In [64]:
trackerTypes = ['BOOSTING', 'MIL', 'KCF','TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']

def createTrackerByName(trackerType):
    
  # Create a tracker based on tracker name
    if trackerType == trackerTypes[0]:
        tracker = cv2.TrackerBoosting_create()
  
    elif trackerType == trackerTypes[1]: 
        tracker = cv2.TrackerMIL_create()
   
    elif trackerType == trackerTypes[2]:
        tracker = cv2.TrackerKCF_create()
  
    elif trackerType == trackerTypes[3]:
        tracker = cv2.TrackerTLD_create()
   
    elif trackerType == trackerTypes[4]:
        tracker = cv2.TrackerMedianFlow_create()
    
    elif trackerType == trackerTypes[5]:
        tracker = cv2.TrackerGOTURN_create()
            
    elif trackerType == trackerTypes[6]:
        tracker = cv2.TrackerMOSSE_create()
        
    elif trackerType == trackerTypes[7]:
        tracker = cv2.TrackerCSRT_create()
        
    else:
        tracker = None
        print('Incorrect tracker name')
        print('Available trackers are:')
        for t in trackerTypes:
            print(t)
     
    return tracker


In [65]:

def get_direction(videoPath, trackerType, boxes_list, frame_no):

    '''
    Detects direction of each vehicle in the video and returns and a list 
    containing these directions

    Parameters:
    videoPath(string) : path of the video
    trackerType(string) : the openCV tracker type to be used  
    boxes_list(list) : list containing bounding boxes for all detected vehicles 
    frame_no : frame no of the video for which you require directions

    '''
    k = 0 #iterator for frame no
    multiTracker_back = cv2.MultiTracker_create()
    #multiTracker = cv2.MultiTracker_create()
    n = len(boxes_list)
    directions = [0]*n
    x1 = [0]*n  #x coordinates of vehicles in frame_no
    x2 = [0]*n  #x coordinates of vehicles in frame_no - 5

    #reading the video
    cap = cv2.VideoCapture(videoPath)
    success, frame = cap.read()
    if not success:
        print('Failed to read video')
        sys.exit(1)

    #the direction detection logic starts here
    while cap.isOpened() :
        success, frame = cap.read()
        if not success:
            break
        
        #saving past few frames as we will apply traking in backward direction
        #form the frame_no
        # if( k == frame_no -10 ):
        #     frame_b10 = frame
        # if ( k == frame_no - 9):
        #     frame_b9 = frame
        # if ( k == frame_no - 8):
        #     frame_b8 = frame
        # if ( k == frame_no - 7):
        #     frame_b7 = frame
        # if ( k == frame_no - 6):
        #     frame_b6 = frame
        if ( k == frame_no - 5):
            frame_b5 = frame
        if ( k == frame_no - 4):
            frame_b4 = frame
        if ( k == frame_no - 3):
            frame_b3 = frame
        if ( k == frame_no - 2):
            frame_b2 = frame
        if ( k == frame_no - 1):
            frame_b1 = frame


        if k == frame_no:
            #boxes_list = [tuple(l) for l in boxes_list]
            for i in range(n):
                box = boxes_list[i]
                x1[i] = (box[0]+2*box[2])/2
                multiTracker_back.add(createTrackerByName(trackerType), frame, box)
            
            #all the vehicles from boxes_list added in the traker instance
            #applying vehicle tracking in backward direction
                
            (success_back, boxes_back) = multiTracker_back.update(frame_b1)
            (success_back, boxes_back) = multiTracker_back.update(frame_b2)
            (success_back, boxes_back) = multiTracker_back.update(frame_b3)
            (success_back, boxes_back) = multiTracker_back.update(frame_b4)
            (success_back, boxes_back) = multiTracker_back.update(frame_b5)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b6)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b7)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b8)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b9)
            #(success_back, boxes_back) = multiTracker_back.update(frame_b10)


            for i in range(n):
                    b = boxes_back[i]
                    x2[i] = (b[0]+2*b[2])/2
                    #print('x1',i,x1[i])


            for i in range(n):
                
                x_d =  x2[i] - x1[i]
                #print(x_d)
                if x_d > 0:
                    directions[i] = 1
                elif x_d < 0:
                    directions[i] = -1

            for i in range(n):
                box = boxes_list[i]
                (x,y,w,h) = [int(v) for v in box]
                if(directions[i] == 0): # not moving
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
                elif(directions[i] == 1): # direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
                elif(directions[i] == -1): # opposite to direction of interest
                    cv2.rectangle(frame, (x,y), (x+w,y+h), (0,0,255), 2)

            #cv2.imshow('frame', frame)
            cv2.waitKey(0)
            break
        k = k+1
        
    cv2.destroyAllWindows()
    return np.array(directions)
        


In [66]:
def find_region(x,y):

    '''
    Returns the region in which the particular point lies

    Parameters:
    x(int) : x coordinate of the point
    y(int) : y coordinate of the point

    Returns:
    int : region no (1/2/3/4/5/6)
    '''


    video_height = 1080
    video_width = 1920

    if(x<640 and y>=540):
        return 1 #Bottom left
    if(x<=1280 and y>=540):
        return 2 #Bottom center
    if(x>1280 and y>=540):
        return 3 #Bottom right
    if(x>1280 and y<540):
        return 4 #Top right
    if(x<=1280 and y<540):
        return 5 #Top center
    if(x<640 and y<540):
        return 6 #Top left

def extract_features_from_frame(bounding_boxes, frame_no, videoPath, trackerType = "KCF"):
        
    '''
    Extract features from a particular frame

    Parameters:
    bounding_boxes(array) : detection array of the video
    frame_no(int) : frame no for which features are to be extracted
    vname(string) : path of the video

    Returns:
    list : containing 18 features of the frame
    '''    

    no_of_boxes = len(bounding_boxes[frame_no])
    boxes_frame = np.array(bounding_boxes[frame_no])

    temp = bounding_boxes[frame_no]
    boxes_list = [(b[0],b[1],b[2]-b[0],b[3]-b[1]) for b in temp]

    directions = get_direction(videoPath, trackerType, boxes_list, frame_no)

    if len(directions) != no_of_boxes:
        raise Exception('mismatch in number of boxes')

    indices_required_boxes = np.where(directions>=0)[0]
    no_of_boxes_required = len(indices_required_boxes)

    boxes_required = boxes_frame[indices_required_boxes]

    #feature - number of vehicles in each region
    r1 = r2 = r3 = r4 = r5 = r6 = 0
    
    #feature - total area of vehicles in each region
    r1_area = r2_area = r3_area = r4_area = r5_area = r6_area = 0

    # for distance calculation for every region
    bottom_center_point_x = 1920/2    # width/2
    bottom_center_point_y = 1080      # height
    r1_min_distance = r2_min_distance = r3_min_distance = r4_min_distance = r5_min_distance = r6_min_distance = 0

    for j in range(len(boxes_required)):
        # x coordinate of center of bounding box cx
        cx = (boxes_required[j][0] + boxes_required[j][2])/2

        # y coordinate of center of bounding box cy
        cy = (boxes_required[j][1] + boxes_required[j][3])/2

        # area of bounding box
        bounding_box_area = ((boxes_required[j][3]-boxes_required[j][1])*
                        (boxes_required[j][2]-boxes_required[j][0]))

        # distance calculation for every bounding box
        distance = sqrt((cx - bottom_center_point_x)**2 + (cy - bottom_center_point_y)**2)

        if(find_region(cx,cy)==1):
            r1 = r1 + 1
            r1_area = r1_area + bounding_box_area
            if(r1_min_distance==0 or distance < r1_min_distance):
                r1_min_distance = distance
        elif(find_region(cx,cy)==2):
            r2 = r2 + 1
            r2_area = r2_area + bounding_box_area
            if(r2_min_distance==0 or distance < r2_min_distance):
                r2_min_distance = distance
        elif(find_region(cx,cy)==3):
            r3 = r3 + 1
            r3_area = r3_area + bounding_box_area
            if(r3_min_distance==0 or distance < r3_min_distance):
                r3_min_distance = distance
        elif(find_region(cx,cy)==4):
            r4 = r4 + 1
            r4_area = r4_area + bounding_box_area
            if(r4_min_distance==0 or distance < r4_min_distance):
                r4_min_distance = distance
        elif(find_region(cx,cy)==5):
            r5 = r5 + 1
            r5_area = r5_area + bounding_box_area
            if(r5_min_distance==0 or distance < r5_min_distance):
                r5_min_distance = distance
        elif(find_region(cx,cy)==6):
            r6 = r6 + 1
            r6_area = r6_area + bounding_box_area
            if(r6_min_distance==0 or distance < r6_min_distance):
                r6_min_distance = distance
                
    features_frame = [r1,r1_area,r1_min_distance,r2,r2_area,r2_min_distance,r3,r3_area,r3_min_distance,r4,r4_area,r4_min_distance,r5,r5_area,r5_min_distance,r6,r6_area,r6_min_distance]
        
    return features_frame

In [67]:
def get_labels_from_video(findex, no_frames, labels_csv):
    
    '''
    Get labels for a particular video 

    Parameters:
    findex(int) : video number in labels_csv for which labels are to be generated(starting from 0)
    no_frames(int) : no of frames in the given video
    labels_csv : the manually marked labels

    Returns:
    list : list with len = no of frames and the value at each index represents safe/unsafe at that frame_no (frame_no starting at 0)
    int : -1 if there is no safe duration in video, 1 otherwise
    '''

    labels = [0]*no_frames
    #print(findex,no_frames)
    if(np.isnan(labels_csv[findex][0])):
        return labels,-1 #there is no safe duration in the given video so all labels marked 0
    else:
        j = 0
        while(j<len(labels_csv[findex]) and not(np.isnan(labels_csv[findex][j]))):
            #mapping safe_start and safe_end in seconds to frame no
            #not that frame no starts with 0
            safe_start = max(int(labels_csv[findex][j]*30)-1,0) 
            safe_end = min(int(labels_csv[findex][j+1]*30)-1, no_frames-1)
            labels[safe_start:safe_end+1] = [1]*(safe_end-safe_start+1) #marking the value b/w safe_start and safe_end with 1
            j = j+2
    if len(labels) > no_frames: #len of labels cannot be greater than no_frames in video
        raise Exception('Check the labels assigned in CSV file!')
    return labels,1


In [68]:
# alternate generate_datafrmae function with frames selected from unsafe videos
def generate_dataframe(arrays_folder, videos_folder, findex, labels_csv):

    '''
    Generate dataframes for model training

    Parameters:
    arrays_folder(path) : folder containing detection arrays
    videos_folder(path) : folder containing videos
    findex(int) : array number to start with from the given folder (starting from 0)
    labels_csv : the manually marked labels

    Returns:
    features_dataframe : dataframe containg 18 features per frame
    labels_dataframe : dataframe containing 1 label per frame
    '''
    
    #getting all paths of all arrays in a list in proper order 
    arrays = glob.glob(arrays_folder+'/array*.npy')
    arrays = natsort.natsorted(arrays)
    videos = glob.glob(videos_folder+'/video*.MOV')
    videos = natsort.natsorted(videos)

    cols_f = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance'] #18 features per frame
    cols_l = ['safe/unsafe'] #1 label per frame
    features_dataframe = pd.DataFrame()
    labels_dataframe = pd.DataFrame()

    for (fname, vname) in zip(arrays, videos):
        print("processing ",fname,":")
        bounding_boxes = np.load(fname, allow_pickle=True) #loading the numpy array containing all detected vehicles
        no_frames = bounding_boxes.shape[0]
        print("no of frames: ", no_frames)
        labels, flag = get_labels_from_video(findex, no_frames, labels_csv)
        print('flag', flag)

        labels_0 = [i for i, value in enumerate(labels) if value == 0 and i>6]
        # shuffling unsafe frames
        random.shuffle(labels_0)
        # selecting 50 frames from unsafe videos instead of rejecting the whole video
        if(len(labels_0) >= 50): 
            labels_0 = labels_0[:50]

        if(flag == -1):
            print('no safe time')
            labels_1 = []
        else:
            labels_1 = [i for i, value in enumerate(labels) if value == 1 and i>6]
            # shuffling safe frames
            random.shuffle(labels_1)
            ## selecting no of safe frames = no of unsafe frames from each video ##
            # label 0 frames will be more than label 1 frame, 
            # so we will try to select same number of safe and unsafe frames
            if(len(labels_0) >= len(labels_1)):
                labels_0 = labels_0[:len(labels_1)]

        features = []
            
        # selecting number of safe and unsafe frames for model training
        ####### to be changed after thinking
        # selecting only 50 frames of each label from a video
        # random.shuffle(labels_0)
        # random.shuffle(labels_1)
        # if(len(labels_0) >= 50): 
        #     labels_0 = labels_0[:50]
        # if(len(labels_1) >= 50): 
        #     labels_1 = labels_1[:50]  
        ####### 
 

        for frame_no in labels_0:
            features_frame = extract_features_from_frame(bounding_boxes, frame_no, vname)
            features.append(features_frame)
        for frame_no in labels_1:
            features_frame = extract_features_from_frame(bounding_boxes, frame_no, vname)
            features.append(features_frame)
        
        #generating and returning features and labels dataframes
        no_frames_in_features = len(features)
        print("no_frames_in_features",no_frames_in_features)
        labels = [0]*len(labels_0) + [1]*len(labels_1)
        print("no_of_frames_in_labels", len(labels))
        findex = findex + 1
        df1 = pd.DataFrame(features, columns=cols_f)
        features_dataframe = features_dataframe.append(df1,ignore_index=True)
        print("df1.size",df1.shape)
        df2 = pd.DataFrame(labels, columns=cols_l)
        labels_dataframe = labels_dataframe.append(df2,ignore_index=True)
        print("df2.size",df2.shape)
        
    return features_dataframe, labels_dataframe

In [69]:
from numpy import genfromtxt

labels_train = genfromtxt(path_train_labels, delimiter =',')
labels_test = genfromtxt(path_test_labels, delimiter =',')
print(labels_train.shape, labels_test.shape)

(60, 4) (16, 6)


In [60]:
features_df_train, labels_df_train = generate_dataframe(path_train_arrays,path_train_videos,0,labels_train)
# features_df_train.to_pickle('features_df_train.pkl')
# labels_df_train.to_pickle('labels_df_train.pkl')

features_df_test, labels_df_test = generate_dataframe(path_test_arrays,path_test_videos, 0,labels_test)
# features_df_test.to_pickle('features_df_test.pkl')
# labels_df_test.to_pickle('labels_df_test.pkl')
print(features_df_train.shape)

processing  /home/siddhi/Desktop/RoadCrossingAssistant_FY_Project_Data/arrays_train_v2/array1.npy :
no of frames:  210
flag -1
no safe time


UnboundLocalError: local variable 'frame_b3' referenced before assignment

## Feature scaling
Strategies that can be used: 
### 1. MinMaxScaler
### 2. StandardScaler
### 3. Unit Vector Scaler

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# StandardScaler
# feature_scaler = StandardScaler()

# MinMaxScaler
feature_scaler = MinMaxScaler()

features_train_scaled = pd.DataFrame(feature_scaler.fit_transform(features_df_train))
features_train_scaled.columns = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance']
features_test_scaled = pd.DataFrame(feature_scaler.transform(features_df_test))
features_test_scaled.columns = ['region1','region1_area','region1_min_distance','region2','region2_area','region2_min_distance','region3','region3_area','region3_min_distance','region4','region4_area','region4_min_distance','region5','region5_area','region5_min_distance','region6','region6_area','region6_min_distance']

In [24]:
# scaled dataset

model = SVC(random_state=0)
model.fit(features_train_scaled, labels_df_train) 
from sklearn.metrics import accuracy_score
 
# print prediction results 
print("for train data:")
predictions = model.predict(features_train_scaled) 
print(classification_report(labels_df_train, predictions)) 
print("accuracy score", accuracy_score(labels_df_train, predictions))

print("\nfor test data:")
predictions = model.predict(features_test_scaled) 
print(classification_report(labels_df_test, predictions)) 
print("accuracy score",accuracy_score(labels_df_test, predictions))


for train data:
              precision    recall  f1-score   support

           0       0.76      0.67      0.71      2939
           1       0.73      0.81      0.77      3280

    accuracy                           0.74      6219
   macro avg       0.75      0.74      0.74      6219
weighted avg       0.75      0.74      0.74      6219

accuracy score 0.7440102910435762

for test data:
              precision    recall  f1-score   support

           0       0.42      0.70      0.53       729
           1       0.72      0.44      0.55      1245

    accuracy                           0.54      1974
   macro avg       0.57      0.57      0.54      1974
weighted avg       0.61      0.54      0.54      1974

accuracy score 0.5390070921985816
